In [1]:
import os
import glob
import numpy as np
import tensorflow as tf
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [2]:
def load_data(dataset_path, img_size=(128, 128)):
    """
    Loads precomputed mel-spectrogram images and extracts labels from folder names.
    """
    X = []
    y = []
    class_names = sorted(os.listdir(dataset_path))  # Get emotion categories
    
    for label in class_names:
        class_path = os.path.join(dataset_path, label)
        if not os.path.isdir(class_path):
            continue
        
        files = glob.glob(os.path.join(class_path, '*.png'))
        print(f"Found {len(files)} images for class '{label}'.")
        
        for file in files:
            img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)  # Load in grayscale
            img = cv2.resize(img, img_size)  # Resize to standard size
            X.append(img)
            y.append(label)
    
    X = np.array(X, dtype=np.float32) / 255.0  # Normalize pixel values
    y = np.array(y)
    return X, y

In [3]:
def preprocess_labels(y):
    """Encodes string labels into numerical one-hot vectors."""
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    y_onehot = to_categorical(y_encoded)
    return y_onehot, le

In [4]:
def build_model(input_shape, num_classes):
    """Builds and returns a CNN model."""
    model = Sequential()
    
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.3))
    
    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.4))
    
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    
    return model

In [5]:
def scheduler(epoch, lr):
    return lr * 0.5 if epoch > 0 and epoch % 100 == 0 else lr

In [6]:
def main():
    dataset_path = 'RAVDESS_mel_spectrograms'  # Update with your dataset path
    X, y = load_data(dataset_path)
    X = X[..., np.newaxis]  # Add channel dimension
    
    y_onehot, le = preprocess_labels(y)
    
    X_train, X_val, y_train, y_val = train_test_split(
        X, y_onehot, test_size=0.2, random_state=42, stratify=np.argmax(y_onehot, axis=1)
    )
    
    input_shape = X_train.shape[1:]
    num_classes = y_onehot.shape[1]
    model = build_model(input_shape, num_classes)
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    early_stop = EarlyStopping(monitor='val_loss', patience=200, restore_best_weights=True)
    lr_scheduler = LearningRateScheduler(scheduler)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=100, min_lr=1e-6)
    
    model.fit(X_train, y_train, epochs=2000, batch_size=32, validation_data=(X_val, y_val),
              callbacks=[early_stop, lr_scheduler, reduce_lr])
    
    loss, acc = model.evaluate(X_val, y_val)
    print(f"Validation Loss: {loss:.4f}, Validation Accuracy: {acc:.4f}")

In [7]:
if __name__ == '__main__':
    main()

Found 192 images for class 'angry'.
Found 192 images for class 'calm'.
Found 192 images for class 'disgust'.
Found 192 images for class 'fearful'.
Found 192 images for class 'happy'.
Found 96 images for class 'neutral'.
Found 192 images for class 'sad'.
Found 192 images for class 'surprised'.
Epoch 1/2000
36/36 [==============================] - 1s 23ms/step - loss: 2.0549 - accuracy: 0.1649 - val_loss: 1.9798 - val_accuracy: 0.2639 - lr: 0.0010
Epoch 2/2000
36/36 [==============================] - 1s 17ms/step - loss: 1.8161 - accuracy: 0.2995 - val_loss: 1.6218 - val_accuracy: 0.3819 - lr: 0.0010
Epoch 3/2000
36/36 [==============================] - 1s 17ms/step - loss: 1.5887 - accuracy: 0.4106 - val_loss: 1.4745 - val_accuracy: 0.4653 - lr: 0.0010
Epoch 4/2000
36/36 [==============================] - 1s 18ms/step - loss: 1.4289 - accuracy: 0.4653 - val_loss: 1.4689 - val_accuracy: 0.4271 - lr: 0.0010
Epoch 5/2000
36/36 [==============================] - 1s 17ms/step - loss: 1.2573 